<a href="https://colab.research.google.com/github/thaisrezendeb/ufes_machineLearning/blob/main/Exercicio8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**8) Usando as técnicas de seleção de características SFS e SBS sobre a base de dados Cardiotocography (disponível em http://archive.ics.uci.edu/ml/datasets/Cardiotocography) (considere o problema de classificar em 3 classes e remova a coluna da classificação em 10 classes) faça:**

**a) Divida a base de dados em três partes de forma estratificada. Selecione 6 atributos usando uma parte da base de dados como treinamento e valide os atributos sobre uma outra parte usando a métrica acurácia. Após determinar os 6 atributos, obtenha a acurácia sobre a terceira parte, usando as duas partes anteriores como treinamento. Use o classificador Vizinho mais Próximo nesta tarefa. Quais foram os atributos
selecionados?**

OBS: O arquivo CTG.xls teve que ser editado por não apresentar um formato válido para importação.

In [ ]:
import numpy as np
import pandas as pd

pd.set_option('mode.chained_assignment', None)

ctg = pd.read_csv("CTG.csv", sep=";", decimal=",")

#Estratificação das amostras
ctg1 = ctg[ctg['NSP'] == 1]
ctg2 = ctg[ctg['NSP'] == 2]
ctg3 = ctg[ctg['NSP'] == 3]

ctg1s = round(ctg1.shape[0]/3)
ctg2s = round(ctg2.shape[0]/3)
ctg3s = round(ctg3.shape[0]/3)

ctg_train1 = pd.concat([ctg1.iloc[:ctg1s], ctg2.iloc[:ctg2s], ctg3.iloc[:ctg3s]])
ctg_train2 = pd.concat([ctg1.iloc[ctg1s:ctg1s*2], ctg2.iloc[ctg2s:ctg2s*2], ctg3.iloc[ctg3s:ctg3s*2]])
ctg_test = pd.concat([ctg1.iloc[ctg1s*2:], ctg2.iloc[ctg2s*2:], ctg3.iloc[ctg3s*2:]])

In [ ]:
def vizinho(base_train, base_test):
  acuracia = 0
  
  #Cálculo da distância Euclidiana
  #Verifica a distância de cada amostra da base de teste com cada uma das amostras da base de treinamento
  for i, row in base_test.iterrows():
    dist_min = 1000000
    classif_new = row[base_test.shape[1]-1]

    for j, row_next in base_train.iterrows():
      dist = 0
      for k in range(0, base_test.shape[1]-1):
        #print(k)
        dist = dist + np.power(row_next[k]-row[k],2)

      dist = np.sqrt(dist)
      if dist < dist_min:
        dist_min = dist
        classif_new = row_next[base_test.shape[1]-1]
    
    if row[base_test.shape[1]-1] == classif_new:
      acuracia = acuracia + 1

  #  print("Classif original de", i, ":", row[6], "nova classif", classif_new)
  acuracia = np.divide(acuracia, base_test.shape[0])

  return acuracia

In [ ]:
###############
#Algoritmo SFS
###############
def sfs(base_train, base_test, attrib):
  max_elem = 0
  max_ac = 0
  idx = base_train.columns

  for n in range(0, base_train.shape[1]-1):    
    if attrib.size > 0:
      #Verifica se atributo já foi inserido
      if np.count_nonzero(attrib == idx[n]) > 0:
        #Atributo já inserido, pula para o próximo
        continue

    #Inicia base com atributos anteriormente analisados
    df_train = base_train[(attrib)]
    df_test = base_test[(attrib)]
  
    #Adiciona atributo para teste
    df_train[idx[n]] = base_train.iloc[:,n]
    df_test[idx[n]] = base_test.iloc[:,n]

    #Adiciona atributo de classificação
    df_train[idx[-1]] = base_train.iloc[:,-1]
    df_test[idx[-1]] = base_test.iloc[:,-1]

    ac = vizinho(df_train, df_test)
    print("Acurácia com {} elemento(s) {} + [{}]: {:.2f}%" \
          .format(attrib.size+1, \
                  attrib, \
                  idx[n], \
                  ac*100))
    
    if ac > max_ac:
      max_ac = ac
      max_elem = n
  
  #Adiciona elemento ao array
  attrib = np.append(attrib, idx[max_elem])

  return attrib

In [ ]:
#SFS inicia com conjunto vazio de atributos
atrib_sfs = np.array([])

while atrib_sfs.size < 6:
  atrib_sfs = sfs(ctg_train1, ctg_train2, atrib_sfs)

Acurácia com 1 elemento(s) [] + [LB]: 77.86%
Acurácia com 1 elemento(s) [] + [AC]: 76.16%
Acurácia com 1 elemento(s) [] + [FM]: 78.56%
Acurácia com 1 elemento(s) [] + [UC]: 72.50%
Acurácia com 1 elemento(s) [] + [DL]: 73.48%
Acurácia com 1 elemento(s) [] + [DS]: 77.86%
Acurácia com 1 elemento(s) [] + [DP]: 79.97%
Acurácia com 1 elemento(s) [] + [ASTV]: 85.33%
Acurácia com 1 elemento(s) [] + [MSTV]: 80.25%
Acurácia com 1 elemento(s) [] + [ALTV]: 79.55%
Acurácia com 1 elemento(s) [] + [MLTV]: 77.15%
Acurácia com 1 elemento(s) [] + [Width]: 78.28%
Acurácia com 1 elemento(s) [] + [Min]: 74.61%
Acurácia com 1 elemento(s) [] + [Max]: 77.29%
Acurácia com 1 elemento(s) [] + [Nmax]: 77.86%
Acurácia com 1 elemento(s) [] + [Nzeros]: 77.86%
Acurácia com 1 elemento(s) [] + [Mode]: 73.91%
Acurácia com 1 elemento(s) [] + [Mean]: 81.95%
Acurácia com 1 elemento(s) [] + [Median]: 81.24%
Acurácia com 1 elemento(s) [] + [Variance]: 78.00%
Acurácia com 1 elemento(s) [] + [Tendency]: 77.86%
Acurácia com 2 e

In [ ]:
#Acurácia sobre o total de amostras considerando os atributos selecionados
ctg_train = pd.concat([ctg_train1, ctg_train2])
ctg_train = ctg_train[(atrib_sfs)]
ctg_test = ctg_test[(atrib_sfs)]

ac_sfs = vizinho(ctg_train, ctg_test)
print("Acurácia sobre total de amostras considerando atributos por SFS: {:.2f}%" \
      .format(ac_sfs*100))
print("Atributos selecionados: {}".format(atrib_sfs))

Acurácia sobre total de amostras considerando atributos por SFS: 35.88%
Atributos selecionados: ['ASTV', 'MSTV', 'AC', 'DS', 'DP', 'DL']


In [ ]:
###############
#Algoritmo SBE
###############
def sbe(base_train, base_test, attrib):
  max_ac = 0
  max_elem = 0

  idx = attrib[:-1]
  print("\nConjunto original ({} elementos): {}".format(idx.size, idx))

  for n in range(0, attrib.size-1):
    #Inicia base com atributos completos
    df_train = base_train[(attrib)]
    df_test = base_test[(attrib)]

    #Remove atributo
    df_train = df_train.drop(attrib[n], axis=1)
    df_test = df_test.drop(attrib[n], axis=1)
    
    ac = vizinho(df_train, df_test)
    print("Acurácia com {} elementos - removido [{}]: {:.2f}%" \
          .format(idx.size-1, \
                  attrib[n], \
                  ac*100))
    
    if ac > max_ac:
      max_ac = ac
      max_elem = n
  
  attrib = np.delete(attrib, max_elem)

  return attrib

In [ ]:
#Conjunto para SBE inicia completo
atrib_sbe = ctg.columns

while atrib_sbe.size > 7: #considerando o atributo de classificação
  atrib_sbe = sbe(ctg_train1, ctg_train2, atrib_sbe)


Conjunto original (21 elementos): Index(['LB', 'AC', 'FM', 'UC', 'DL', 'DS', 'DP', 'ASTV', 'MSTV', 'ALTV',
       'MLTV', 'Width', 'Min', 'Max', 'Nmax', 'Nzeros', 'Mode', 'Mean',
       'Median', 'Variance', 'Tendency'],
      dtype='object')
Acurácia com 20 elementos - removido [LB]: 82.65%
Acurácia com 20 elementos - removido [AC]: 82.65%
Acurácia com 20 elementos - removido [FM]: 82.65%
Acurácia com 20 elementos - removido [UC]: 82.65%
Acurácia com 20 elementos - removido [DL]: 82.65%
Acurácia com 20 elementos - removido [DS]: 82.65%
Acurácia com 20 elementos - removido [DP]: 82.65%
Acurácia com 20 elementos - removido [ASTV]: 79.69%
Acurácia com 20 elementos - removido [MSTV]: 82.65%
Acurácia com 20 elementos - removido [ALTV]: 82.93%
Acurácia com 20 elementos - removido [MLTV]: 81.52%
Acurácia com 20 elementos - removido [Width]: 81.81%
Acurácia com 20 elementos - removido [Min]: 83.50%
Acurácia com 20 elementos - removido [Max]: 82.23%
Acurácia com 20 elementos - removido [Nmax]

In [ ]:
#Acurácia sobre o total de amostras considerando os atributos selecionados
ctg_train = pd.concat([ctg_train1, ctg_train2])
ctg_train = ctg_train[(atrib_sbe)]
ctg_test = ctg_test[(atrib_sbe)]

ac_sbe = vizinho(ctg_train, ctg_test)
print("Acurácia sobre total de amostras considerando atributos por SBE: {:.2f}%" \
      .format(ac_sbe*100))
print("Atributos selecionados: {}".format(atrib_sbe[:-1]))

Acurácia sobre total de amostras considerando atributos por SBE: 59.46%
Atributos selecionados: Index(['ASTV', 'MLTV', 'Width', 'Max', 'Mode', 'Mean'], dtype='object')


**b) Realize o mesmo procedimento de a), mas agora selecionando os atributos usando duas partes para treinamento e validando sobre as mesmas duas partes. Após determinar os atributos, obtenha a acurácia sobre a terceira parte. A acurácia sobre a terceira parte foi melhor, igual ou pior do que a obtida na letra a)? Esse era o resultado esperado? Esse procedimento parece correto? Por quê?**

In [ ]:
#Seleção de atributos por SFS
ctg_train = pd.concat([ctg_train1, ctg_train2])

atrib_sfs2 = np.array([])

while atrib_sfs2.size < 6:
  atrib_sfs2 = sfs(ctg_train, ctg_train, atrib_sfs2)

Acurácia com 1 elemento(s) [] + [LB]: 78.28%
Acurácia com 1 elemento(s) [] + [AC]: 80.32%
Acurácia com 1 elemento(s) [] + [FM]: 87.66%
Acurácia com 1 elemento(s) [] + [UC]: 84.77%
Acurácia com 1 elemento(s) [] + [DL]: 82.65%
Acurácia com 1 elemento(s) [] + [DS]: 77.93%
Acurácia com 1 elemento(s) [] + [DP]: 81.66%
Acurácia com 1 elemento(s) [] + [ASTV]: 86.25%
Acurácia com 1 elemento(s) [] + [MSTV]: 81.66%
Acurácia com 1 elemento(s) [] + [ALTV]: 82.58%
Acurácia com 1 elemento(s) [] + [MLTV]: 78.14%
Acurácia com 1 elemento(s) [] + [Width]: 79.55%
Acurácia com 1 elemento(s) [] + [Min]: 76.87%
Acurácia com 1 elemento(s) [] + [Max]: 77.64%
Acurácia com 1 elemento(s) [] + [Nmax]: 77.86%
Acurácia com 1 elemento(s) [] + [Nzeros]: 78.00%
Acurácia com 1 elemento(s) [] + [Mode]: 78.98%
Acurácia com 1 elemento(s) [] + [Mean]: 80.39%
Acurácia com 1 elemento(s) [] + [Median]: 80.25%
Acurácia com 1 elemento(s) [] + [Variance]: 79.41%
Acurácia com 1 elemento(s) [] + [Tendency]: 77.86%
Acurácia com 2 e

In [ ]:
ctg_train = ctg_train[(atrib_sfs2)]
ctg_test = ctg_test[(atrib_sfs2)]
ac_sfs2 = vizinho(ctg_train, ctg_test)
print("Acurácia total com atributos por SFS: {:.2f}%".format(ac_sfs2*100))
print("Atributos selecionados: {}".format(atrib_sfs2))

Acurácia total com atributos por SFS: 1.69%
Atributos selecionados: ['FM' 'ASTV' 'MLTV' 'LB' 'AC' 'UC']


In [ ]:
#Seleção de atributos por SBE
ctg_train = pd.concat([ctg_train1, ctg_train2])

atrib_sbe2 = ctg_train.columns

while atrib_sbe2.size > 7: #considerando coluna de classificação
  atrib_sbe2 = sbe(ctg_train, ctg_train, atrib_sbe2)


Conjunto original (14 elementos): ['ASTV' 'MSTV' 'ALTV' 'MLTV' 'Width' 'Min' 'Max' 'Nmax' 'Nzeros' 'Mode'
 'Mean' 'Median' 'Variance' 'Tendency']
Acurácia com 13 elementos - removido [ASTV]: 100.00%
Acurácia com 13 elementos - removido [MSTV]: 100.00%
Acurácia com 13 elementos - removido [ALTV]: 100.00%
Acurácia com 13 elementos - removido [MLTV]: 100.00%
Acurácia com 13 elementos - removido [Width]: 100.00%
Acurácia com 13 elementos - removido [Min]: 100.00%
Acurácia com 13 elementos - removido [Max]: 100.00%
Acurácia com 13 elementos - removido [Nmax]: 100.00%
Acurácia com 13 elementos - removido [Nzeros]: 100.00%
Acurácia com 13 elementos - removido [Mode]: 100.00%
Acurácia com 13 elementos - removido [Mean]: 100.00%
Acurácia com 13 elementos - removido [Median]: 100.00%
Acurácia com 13 elementos - removido [Variance]: 100.00%
Acurácia com 13 elementos - removido [Tendency]: 100.00%

Conjunto original (13 elementos): ['MSTV' 'ALTV' 'MLTV' 'Width' 'Min' 'Max' 'Nmax' 'Nzeros' 'Mode' 

Obs: resultado está truncado devido ao elevado tempo de execução e à limitação do ambiente do Colab. Todos os resultados de acurácia anteriormente obtidos foram de 100% para todas as combinações de conjuntos de atributos.

In [ ]:
ctg_train = ctg_train[(atrib_sbe2)]
ctg_test = ctg_test[(atrib_sbe2)]
ac_sbe2 = vizinho(ctg_train, ctg_test)
print("Acurácia total com atributos por SBE: {:.2f}%".format(ac_sbe2*100))
print("Atributos selecionados: {}".format(atrib_sbe2[:-1]))

Acurácia total com atributos por SBE: 71.05%
Atributos selecionados: ['MLTV' 'Mode' 'Mean' 'Median' 'Variance' 'Tendency']


A acurácia obtida tanto através da inclusão de atributos (SFS) quanto pela remoção (SBE) é fortemente influenciada pela ordem adotada na seleção. Para este caso foi escolhida a ordem em que aparecem na base de dados, tanto para inclusão quanto para exclusão. A acurácia obtida fazendo o teste com dados diferentes para treinamento e teste foi de **35,88%** com atributos escolhidos por SFS e **59,46%** com atributos escolhidos por SBE. Porém, quando se treina e testa com o mesmo conjunto de dados, as acurácias são de **1,69%** e **71,05%**, respectivamente. Esta é uma situação inconcebível, uma vez que o uso dos mesmos dados para treino e teste causa *overfitting*. O tempo de execução também aumenta exponencialmente, uma vez que a base de treinamento é muito maior que a base de teste. O algoritmo SBE mostrou-se mais custoso, demandando maior processamento, como explicado em sala de aula, uma vez que a base de dados possui 21 atributos e foram solicitados apenas 6. Caso o critério de parada não fosse pela quantidade de atributos, mas sim pela mínima acurácia, poucos atributos seriam selecionados com SFS e também poucos seriam removidos por SBE, pois eles convergem rapidamente para 100% de acurácia.

Pode-se concluir que a escolha pela análise dos atributos na ordem em que aparecem na base de dados não é a melhor para este conjunto de dados, sendo que os atributos finais combinados podem representá-los mais eficientemente.

Ainda, conclui-se que o uso das mesmas amostras para treinamento e teste não é um procedimento correto, uma vez que distorce o resultado do algoritmo.